# Configuración de entorno

## Librerias

In [16]:
# Manejo del sistema operativo y tiempo
import os  # Interactuar con el sistema operativo (rutas, variables de entorno, etc.)
import time  # Manejar operaciones relacionadas con el tiempo (pausas, cálculos de tiempo)

# Manejo de mensajería con Twilio
from twilio.rest import Client  # Uso de la API de Twilio, enviar mensajes o realizar llamadas
from twilio_config import *  # (Archivo personalizado) Configuraciones o credenciales de Twilio

# Peticiones HTTP y manejo de datos en la web
import requests  # Peticiones HTTP
from requests import Request, Session  # Peticiones más avanzadas y manejo persistente de sesiones HTTP
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects  # Manejo de errores en conexiones HTTP
from bs4 import BeautifulSoup  # Extracción y análisis de datos de páginas web en formato HTML/XML

# Manejo de datos y análisis
import pandas as pd  # Manipulación y análisis de datos estructurados
import json  # Manipulación de datos en formato JSON (serialización y deserialización)

# Seguimiento y visualización de progreso
from tqdm import tqdm  # Barra de progreso en bucles largos

# Manejo de fechas y tiempos
from datetime import datetime  # Para trabajar con fechas y horas (formateo, cálculos, etc.)

# URL y variables de entorno
from urllib.parse import urlencode  # Construcción y codificación de parámetros en URLs
from dotenv import load_dotenv  # Cargar variables de entorno desde un archivo .env

## Requirements

- pandas==2.0.3
- requests==2.31.0
- tqdm==4.64.1
- python-dotenv==1.0.0
- beautifulsoup4==4.12.2
- twilio==7.18.0

Para instalar todas las dependencias desde el archivo requirements.txt, usa el siguiente comando en la terminal (dentro de tu entorno virtual):
- pip install -r requirements.txt

# Construcción de URL

In [ ]:
# Paso 1: Variables de configuración: Definir la ciudad, clave de la API y URL base de la API
query = 'Medellin'  # Ciudad a consultar
# api_key = "###############################"  # Clave funcional
api_key = api_key_weather  # Clave de la API (Almacenarla como variable de entorno)
base_url = 'http://api.weatherapi.com/v1/forecast.json'  # URL base de la API

# Paso 2: Construcción de la URL: Construir la URL con parámetros codificados
params = {
            'key': api_key,  # Clave de la API
            'q': query,  # Ubicación a consultar
            'days': 1,  # Número de días del pronóstico
            'aqi': 'no',  # Sin datos de calidad del aire
            'alerts': 'no'  # Sin alertas climáticas
         }
url_clima = f"{base_url}?{urlencode(params)}"  # Construcción de la URL para consumir datos de: www.weatherapi.com

# Paso 3: Realizar la solicitud a la API: # Usar requests para obtener datos del clima desde la API
try:
    response = requests.get(url_clima)  # Realizar la solicitud
    response.raise_for_status()  # Lanza error si ocurre algo
    clima_data = response.json()
    print("Datos del clima:", clima_data)
except requests.exceptions.HTTPError as http_err:
    if response.status_code == 403:
        print("Error: La clave de la API no tiene permisos o es inválida.")
    else:
        print(f"HTTP error: {http_err}")
except requests.exceptions.RequestException as e:
    print(f"Error al realizar la solicitud: {e}")

Datos del clima: {'location': {'name': 'Medellin', 'region': 'Antioquia', 'country': 'Colombia', 'lat': 6.2914, 'lon': -75.5361, 'tz_id': 'America/Bogota', 'localtime_epoch': 1731814306, 'localtime': '2024-11-16 22:31'}, 'current': {'last_updated_epoch': 1731814200, 'last_updated': '2024-11-16 22:30', 'temp_c': 15.2, 'temp_f': 59.4, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 81, 'wind_dir': 'E', 'pressure_mb': 1025.0, 'pressure_in': 30.27, 'precip_mm': 0.09, 'precip_in': 0.0, 'humidity': 94, 'cloud': 50, 'feelslike_c': 15.2, 'feelslike_f': 59.4, 'windchill_c': 12.8, 'windchill_f': 55.0, 'heatindex_c': 12.8, 'heatindex_f': 55.0, 'dewpoint_c': 12.6, 'dewpoint_f': 54.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 3.0, 'gust_kph': 4.8}, 'forecast': {'forecastday': [{'date': '2024-11-16', 'date_epoch': 1731715200, 'day': {'maxtemp_c': 20.4, 'maxte

# Construcción de Dataframe

Se construye una función que extrae información específica de los 24 registros horarios proporcionados por WeatherAPI.
Pasos:
1. Extracción de variables clave:
    - fecha: Fecha asociada al registro.
    - hora: Hora específica del registro.
    - condicion: Condición climática (por ejemplo, "Despejado", "Lluvioso").
    - tempe: Temperatura registrada.
    - rain: Cantidad de lluvia.
    - prob_rain: Probabilidad de lluvia.

2. Procesamiento de los datos:
    - Cada una de estas variables se extrae de las líneas previamente obtenidas en la consulta a la API de WeatherAPI.
    - Los registros horarios son procesados uno por uno para extraer la información requerida.

El objetivo es consolidar los datos extraídos en un formato adecuado para su posterior análisis o almacenamiento.

In [18]:
def get_forecast(response, i):
    """
    Extrae información de un registro horario específico de la respuesta de WeatherAPI.
    
    Args:
        response (dict): Respuesta JSON de WeatherAPI.
        i (int): Índice del registro horario (0-23).
    
    Returns:
        tuple: Contiene fecha, hora, condición climática, temperatura, lluvia (1 o 0), y probabilidad de lluvia.
    """
    try:
        # Navegación simplificada
        hourly_data = response['forecast']['forecastday'][0]['hour'][i]
        
        # Extracción de variables
        fecha = hourly_data['time'].split()[0]  # Fecha (YYYY-MM-DD)
        hora = int(hourly_data['time'].split()[1].split(':')[0])  # Hora (0-23)
        condicion = hourly_data['condition']['text']  # Condición climática
        tempe = float(hourly_data['temp_c'])  # Temperatura en °C
        rain = int(hourly_data['will_it_rain'])  # Lluvia (1: Sí, 0: No)
        prob_rain = int(hourly_data['chance_of_rain'])  # Probabilidad de lluvia (%)
        
        # Retorno de los datos en el orden esperado
        return fecha, hora, condicion, tempe, rain, prob_rain

    except (KeyError, IndexError) as e:
        print(f"Error al extraer los datos del índice {i}: {e}")
        return None

In [19]:
if isinstance(response, requests.models.Response):
    response = response.json()  # Convierte el objeto Response a un diccionario JSON

# Creación de lista vacía
datos = []

# Ciclo para iterar sobre los 24 registros horarios
for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour='green'):
    datos.append(get_forecast(response, i))

100%|██████████| 24/24 [00:00<00:00, 24093.66it/s]


In [20]:
datos

[('2024-11-16', 0, 'Fog', 12.4, 1, 89),
 ('2024-11-16', 1, 'Fog', 12.4, 1, 74),
 ('2024-11-16', 2, 'Fog', 12.3, 0, 60),
 ('2024-11-16', 3, 'Fog', 12.1, 0, 60),
 ('2024-11-16', 4, 'Fog', 12.0, 1, 71),
 ('2024-11-16', 5, 'Fog', 11.9, 1, 88),
 ('2024-11-16', 6, 'Mist', 11.9, 1, 82),
 ('2024-11-16', 7, 'Mist', 13.4, 0, 0),
 ('2024-11-16', 8, 'Partly Cloudy ', 15.3, 0, 0),
 ('2024-11-16', 9, 'Patchy rain nearby', 17.8, 1, 74),
 ('2024-11-16', 10, 'Patchy rain nearby', 20.2, 1, 100),
 ('2024-11-16', 11, 'Patchy light drizzle', 20.4, 1, 100),
 ('2024-11-16', 12, 'Light rain shower', 19.4, 1, 100),
 ('2024-11-16', 13, 'Patchy light rain', 18.8, 1, 100),
 ('2024-11-16', 14, 'Light rain shower', 18.5, 1, 100),
 ('2024-11-16', 15, 'Light rain shower', 17.9, 1, 100),
 ('2024-11-16', 16, 'Patchy rain nearby', 17.1, 1, 100),
 ('2024-11-16', 17, 'Light rain shower', 16.0, 1, 100),
 ('2024-11-16', 18, 'Light rain shower', 14.0, 1, 100),
 ('2024-11-16', 19, 'Light rain shower', 13.4, 1, 100),
 ('2024-1

Con los datos obtenidos, construiremos un DataFrame que contendrá las siguientes columnas para organizar y analizar la información:

- Fecha: La fecha correspondiente a cada registro.
- Hora: La hora específica del registro (formato 24 horas).
- Condición: La descripción del estado climático (por ejemplo, "Fog", "Mist").
- Temperatura: La temperatura en grados Celsius registrada.
- Lluvia: Indicador de si hubo lluvia (1: Sí, 0: No).
- Probabilidad de lluvia: El porcentaje de probabilidad de lluvia para esa hora.

Estas columnas nos permitirán estructurar los datos de manera tabular y facilitar su análisis mediante el uso de pandas para operaciones adicionales. El DataFrame se generará con el siguiente comando:

In [21]:
# Crear el DataFrame con los datos extraídos
columnas = ['Fecha', 'Hora', 'Condicion', 'Temperatura', 'Lluvia', 'Probabilidad de Lluvia']
df = pd.DataFrame(datos, columns=columnas)

df

,Fecha,Hora,Condicion,Temperatura,Lluvia,Probabilidad de Lluvia
0,2024-11-16,0,Fog,12.4,1,89
1,2024-11-16,1,Fog,12.4,1,74
2,2024-11-16,2,Fog,12.3,0,60
3,2024-11-16,3,Fog,12.1,0,60
4,2024-11-16,4,Fog,12.0,1,71
5,2024-11-16,5,Fog,11.9,1,88
6,2024-11-16,6,Mist,11.9,1,82
7,2024-11-16,7,Mist,13.4,0,0
8,2024-11-16,8,Partly Cloudy,15.3,0,0
9,2024-11-16,9,Patchy rain nearby,17.8,1,74


Con esto, hemos transformado la información recibida desde el JSON a una estructura más manejable gracias a la librería Pandas. Ahora, si trasladamos este análisis a una lógica de negocio, surgen preguntas claves: 

- ¿Qué estado climático le interesa conocer al cliente?
- ¿Cuándo llueve o cuándo no?

La respuesta adecuada sería: cuando llueve. Sin embargo, no solo es relevante conocer el estado climático, también es crucial identificar la hora específica, ya que esta debe estar relacionada con el horario en el que el cliente presta sus servicios. Por ello, filtraremos los datos para incluir únicamente los registros donde:

1. La probabilidad de lluvia sea mayor a 0.
2. La hora esté dentro del rango de 12:00 PM a 6:00 PM (Modo de ejemplo).

Para lograr esto, construiremos un nuevo DataFrame con los siguientes filtros:

In [22]:
# Filtrar datos con probabilidad de lluvia > 0 y en el rango horario deseado
df_lluvia = df[(df['Probabilidad de Lluvia'] > 0) & (df['Hora'].between(12, 18))]

# Visualizar el resultado filtrado
df_lluvia

,Fecha,Hora,Condicion,Temperatura,Lluvia,Probabilidad de Lluvia
12,2024-11-16,12,Light rain shower,19.4,1,100
13,2024-11-16,13,Patchy light rain,18.8,1,100
14,2024-11-16,14,Light rain shower,18.5,1,100
15,2024-11-16,15,Light rain shower,17.9,1,100
16,2024-11-16,16,Patchy rain nearby,17.1,1,100
17,2024-11-16,17,Light rain shower,16.0,1,100
18,2024-11-16,18,Light rain shower,14.0,1,100


# Mensaje a enviar

In [23]:
# Seleccionar las columnas relevantes
df_reducido = df_lluvia[['Hora', 'Condicion']]

# Convertir el DataFrame reducido a una cadena legible
lluvia_str = df_reducido.to_string(index=False)

# Generar mensaje sin saltos de línea adicionales
mensaje = f"Pronóstico del tiempo {df['Fecha'][0]} en {query}: "
mensaje += ", ".join(
    f"Hora: {row['Hora']}, Condición: {row['Condicion']}"
    for _, row in df_lluvia.iterrows()
)

# Mostrar mensaje
print(mensaje)

Pronóstico del tiempo 2024-11-16 en Medellin: Hora: 12, Condición: Light rain shower, Hora: 13, Condición: Patchy light rain, Hora: 14, Condición: Light rain shower, Hora: 15, Condición: Light rain shower, Hora: 16, Condición: Patchy rain nearby, Hora: 17, Condición: Light rain shower, Hora: 18, Condición: Light rain shower


# Mensaje SMS desde Twilio con Python

In [12]:
# Configuración de las variables necesarias
time.sleep(2)  # Pausa opcional antes del envío
account_sid = twilio_account_sid  # SID de tu cuenta Twilio
auth_token = twilio_auth_token  # Token de autenticación
phone_number_from = phone_number  # Número comprado en Twilio (origen)
phone_number_to = '+573127794012'  # Número de destino (receptor)

# Seleccionar columnas relevantes y limitar filas
df_reducido = df[['Hora', 'Condicion']].head(10)  # Seleccionar hasta 10 filas

# Convertir el DataFrame reducido a una cadena compacta
lluvia_str = df_reducido.to_string(index=False, header=False)

# Construcción del mensaje
mensaje = (
    f"\nHola!\n\nEl pronóstico del tiempo para hoy {df['Fecha'][0]} "
    f"en {query} es:\n\n{lluvia_str}"
)

# Validar el tamaño del mensaje antes de enviarlo
if len(mensaje) > 1600:
    print("El mensaje excede los 1600 caracteres. Reduce los datos seleccionados.")
else:
    # Enviar el mensaje SMS
    try:
        client = Client(account_sid, auth_token)  # Inicialización del cliente Twilio
        message = client.messages.create(
            body=mensaje,  # Contenido del mensaje
            from_=phone_number_from,  # Número de origen
            to=phone_number_to  # Número de destino
        )
        print(f"Mensaje enviado correctamente. SID: {message.sid}")
    except Exception as e:
        print(f"Error al enviar el mensaje: {e}")

Mensaje enviado correctamente. SID: SM581b0c1e7bd7f5e1fbf2dd64c4438024
